# REM API Demo With `requests`

This notebook demonstrates how we can call the REM API to get an estimate
for the savings we can expect if we make an electrical upgrade to a home
at a particular address.

## Imports and Configuration

In [1]:
import requests
from pathlib import Path

In [ ]:
HOST = "https://api.rewiringamerica.org"
REM_ADDRESS_URL = f"{HOST}/api/v1/rem/address"

API_KEY = None  # Put your API key here, or better yet in the file ~/.rwapi/api_key.txt

In [15]:
if API_KEY is None:
    api_key_path = Path.home() / ".rwapi" / "api_key.txt"

    if api_key_path.is_file():
        with open(api_key_path) as f:
            API_KEY = f.read().strip()

## Parameters

Address we are interested in and the upgrade we want to do.

In [16]:
address = '8009 Belmont Ave., Lubbock, TX 79424'
upgrade = "high_eff_hp_elec_backup"
heating_fuel = 'natural_gas'

## Make the Request

In [17]:
headers = {"Authorization": f"Bearer {API_KEY}"}

response = requests.get(
    url=REM_ADDRESS_URL,
    headers=headers,
    params=dict(
        address=address, upgrade=upgrade, heating_fuel=heating_fuel
    )
)

## Pull out the results

We are specifically interested in the total dollar savings.

In [18]:
data = response.json()

In [25]:
annual_savings = -data['fuel_results']['total']['delta']['cost']['mean']['value']

f"Expected annual savings: ${annual_savings:.2f}"

'Expected annual savings: $630.39'